# **Model #1**

### **Load Libraies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Pillow
!pip install Keras-Preprocessing
!pip install ktrain

In [ ]:
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
import ktrain
from ktrain import text
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences

sns.set()
%matplotlib inline

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sklearn
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import cross_val_score

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
# Joblib
import joblib # To save models

In [ ]:
datafolder='/content/drive/MyDrive/Community Help /category classification/Data/'
modelfolder='/content/drive/MyDrive/Community Help /category classification/Model/'
arrayfolder='/content/drive/MyDrive/Community Help /category classification/NumPyArrays/'

### **Data Preparion**

In [ ]:
train_data = pd.read_csv(datafolder+'df_category_model1.csv')
train_data.drop(['Unnamed: 0'],
  axis='columns', inplace=True)
train_data['TITLE']=train_data['TITLE'].fillna("")
train_data.isna().sum()

TITLE                 0
CATEGORY_CLASS        0
CATEGORY_CLASS_NEW    0
dtype: int64

In [ ]:
train_data.head()

,TITLE,CATEGORY_CLASS,CATEGORY_CLASS_NEW
0,Raiders now only team in NFL to go three years...,SPORTS,GENERAL-1
1,Mike Tyson goes full matrix dodging punches (x...,SPORTS,GENERAL-1
2,Blake Griffins custom Air Jordans he designed ...,SPORTS,GENERAL-1
3,How would the Premier League look if clubs cou...,SPORTS,GENERAL-1
4,"Fernando Alonso, McLaren driver to miss Monaco...",SPORTS,GENERAL-1


In [ ]:
train_data.shape

(786717, 3)

In [ ]:
X = []
# the genders
y = []
# lengths of X elements
X_len = []
for index, row in train_data.iterrows():
    X.append(row["TITLE"])
    X_len.append([len(row["TITLE"])])
    y.append(row["CATEGORY_CLASS_NEW"])

In [ ]:
joblib.dump(X_len, arrayfolder+'X_len_Classification.pkl')
joblib.dump(y, arrayfolder+'y_Classification.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_Classification.pkl']

#### **Help Methods & Data Cleaning**

In [ ]:
# preprocessing functions
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)

def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)

# remove words that end with one or more identical letters
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)

def remove_allcaps(text):
    caps = re.compile(r'([^a-z0-9()<>\'`\-]){2,}')
    return caps.sub(r'ALLCAPS', text)

def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    return smiley.sub(r'SMILE', text)

def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# tags Part of Speech (POS), because teh lemmatizer needs it
tag_map = defaultdict(lambda : wn.NOUN)
# wn does a grammatical analysis
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# create lemmatizer
word_Lemmatized = WordNetLemmatizer()

def review_to_words(raw_body):
    # remove html tags
    body_text = BeautifulSoup(raw_body).get_text()
    #letters_only = re.sub("[^a-zA-Z]", " ", body_text)
    # lowercase all text
    words = body_text.lower()
    # remove urls
    text = remove_URL(words)
    # remove numbers
    text = remove_number(text)
    # remove smiles
    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
    text = remove_elongated_words(text)
    words = remove_repeat_punct(text)
    # tokenizes and pass to lemmatizer, which lemmatizes taking tags into account (see before)
    words = word_tokenize(words)
    # we don't remove stop words, because doing it on combination with removing the 40 (trial & error estimated parameter) most utilized words (see below) decreases performance
    #stops = set(stopwords.words("english"))
    #meaningful_words = [w for w in words if not w in stops]
    Final_words = []
    for word, tag in pos_tag(words):
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # returns lemmatized texts as strings
    return( " ".join(Final_words))

In [ ]:
clean_train_comments = [review_to_words(x) for x in X]

<ipython-input-14-1e68d722f9d6>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  body_text = BeautifulSoup(raw_body).get_text()
<ipython-input-14-1e68d722f9d6>:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  body_text = BeautifulSoup(raw_body).get_text()


In [ ]:
joblib.dump(clean_train_comments, arrayfolder+'clean_train_comments_Category_Classification_MLP.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/clean_train_comments_Category_Classification_MLP.pkl']

### **CountVectorizer**

In [ ]:
clean_train_comments=joblib.load(arrayfolder+'clean_train_comments_Category_Classification_MLP.pkl')

In [ ]:
X_len=joblib.load(arrayfolder+'X_len_Classification.pkl')
y=joblib.load(arrayfolder+'y_Classification.pkl')

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             max_features = 2000)
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments).toarray()


print(train_data_features.shape)
# print vocabulary
vocab = vectorizer.get_feature_names_out()
#print(vocab)
import numpy as np

# counts how many times a word appearco
dist = np.sum(train_data_features, axis=0)

##for tag, count in zip(vocab, dist):
    ##print(count, tag)

    # removes the 40 most utilized words
for _ in range(40):
    index = np.argmax(dist)
    train_data_features = np.delete(train_data_features, index, axis = 1)

train_data_features.shape

# np array
s = np.concatenate((train_data_features,np.array(X_len)),axis = 1)

# 5000 rows (one per author),  and 2000-40+1 (X_len) features
print(s.shape)

# un np.array
y = np.array(y)

(786717, 2000)
(786717, 1961)


In [ ]:
joblib.dump(vectorizer, arrayfolder+'count_vect_2000.pkl')
joblib.dump(vocab, arrayfolder+'vocab_2000.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/vocab_2000.pkl']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical,plot_model
le=LabelEncoder()
lbls=le.fit_transform(y)
classes=list(le.classes_)
classes=[str(c) for c in classes]
lbls=to_categorical(lbls,num_classes=len(classes))

In [ ]:
joblib.dump(classes, arrayfolder+'classes.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/classes.pkl']

In [ ]:
joblib.dump(lbls, arrayfolder+'lbls_MLP_4000.pkl')
joblib.dump(s, arrayfolder+'s_4000.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/s_4000.pkl']

In [ ]:
classes

### **MLP Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP.pkl')
s=joblib.load(arrayfolder+'s.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)


In [ ]:
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

((314686, 1961), (78672, 1961), (314686, 5), (78672, 5))

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)

In [ ]:
X_train.shape,X_text.shape,y_train.shape,y_text.shape

((157343, 1961), (94406, 1961), (157343, 5), (94406, 5))

In [ ]:
joblib.dump(X_text, arrayfolder+'X_text_MLP.pkl')
joblib.dump(y_text, arrayfolder+'y_text_MLP.pkl')

joblib.dump(X_valid, arrayfolder+'X_valid_MLP.pkl')
joblib.dump(y_valid, arrayfolder+'y_valid_MLP.pkl')

joblib.dump(X_train, arrayfolder+'X_train_MLP.pkl')
joblib.dump(y_train, arrayfolder+'y_train_MLP.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_train_MLP.pkl']

In [ ]:
# Define MLP Classifier:
## Activation function for the hidden layer: "rectified linear unit function"
## Solver for weight optimization: "stochastic gradient-based optimizer"
## Alpha: regularization parameter
## Learning rate schedule for weight updates: "gradually decreases the learning rate at each time step t using an inverse scaling exponent of power_t"
## Verbose: "True" in order to print progress messages to stdout.
## Early stopping: "True" in order to use early stopping to terminate training when validation score is not improving. It automatically sets aside 10% of training data as validation and terminate training when validation score is not improving by at least tol for n_iter_no_change consecutive epochs.

mlpClf = MLPClassifier(activation= 'relu', solver = 'adam',
                       alpha = 0.05, learning_rate = 'invscaling', verbose = True,
                       early_stopping = True, max_iter = 400, random_state=0)

# Model fit
mlpClf.fit(X_train, y_train)

Iteration 1, loss = 1.85937676
Validation score: 0.517954
Iteration 2, loss = 1.44237125
Validation score: 0.537083
Iteration 3, loss = 1.37325913
Validation score: 0.596949
Iteration 4, loss = 1.33607723
Validation score: 0.628789
Iteration 5, loss = 1.29259620
Validation score: 0.650461
Iteration 6, loss = 1.26862525
Validation score: 0.649889
Iteration 7, loss = 1.27114442
Validation score: 0.656308
Iteration 8, loss = 1.22152890
Validation score: 0.653384
Iteration 9, loss = 1.21315637
Validation score: 0.664125
Iteration 10, loss = 1.19259597
Validation score: 0.679504
Iteration 11, loss = 1.17431695
Validation score: 0.672386
Iteration 12, loss = 1.16803710
Validation score: 0.675691
Iteration 13, loss = 1.17942007
Validation score: 0.698951
Iteration 14, loss = 1.16670476
Validation score: 0.677915
Iteration 15, loss = 1.14380228
Validation score: 0.684080
Iteration 16, loss = 1.13039480
Validation score: 0.673467
Iteration 17, loss = 1.12967839
Validation score: 0.688847
Iterat

MLPClassifier(alpha=0.05, early_stopping=True, learning_rate='invscaling',
              max_iter=400, random_state=0, verbose=True)

In [ ]:
joblib.dump(mlpClf, modelfolder+'mlpClf_category.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/mlpClf_category.pkl']

In [ ]:
# Prediction
y_score = mlpClf.predict_proba(X_valid)

In [ ]:
y_score.shape

(78672, 5)

In [ ]:
y_valid.shape

(78672, 5)

In [ ]:
roc_auc = roc(y_valid,y_score, multi_class='ovr')
roc_auc

0.9434414940967637

In [ ]:
ytest_score = mlpClf.predict_proba(X_text)

In [ ]:
print(roc(y_text,ytest_score, multi_class='ovr'))

0.9440865639227572


### **LogisticRegression Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000.pkl')
s=joblib.load(arrayfolder+'s_4000.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

((314686, 3961), (78672, 3961), (314686, 5), (78672, 5))

In [ ]:
lbls=""
s=""

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

((157343, 3961), (94406, 3961), (157343, 5), (94406, 5))

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)
y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)

In [ ]:
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

((157343,), (94406,))

In [ ]:
lr = LogisticRegression(C = 3, max_iter = 1000, n_jobs=-1,multi_class='multinomial')
lr.fit(X_train, y_new)
y_pred1 = lr.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred1)))
print(classification_report(y_test_new, y_pred1))

Accuracy: 0.7843251488252865
              precision    recall  f1-score   support

           1       0.76      0.79      0.77     19699
           2       0.79      0.87      0.83     20698
           3       0.78      0.72      0.75     17977
           4       0.80      0.76      0.78     18782
           5       0.80      0.77      0.78     17250

    accuracy                           0.78     94406
   macro avg       0.78      0.78      0.78     94406
weighted avg       0.78      0.78      0.78     94406



In [ ]:
joblib.dump(lr, modelfolder+'LogisticRegression_category_4000.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/LogisticRegression_category_4000.pkl']

###  **Support Vector Machine**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000.pkl')
s=joblib.load(arrayfolder+'s_4000.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)

y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_new)
y_pred2 = svc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred2)))
print(classification_report(y_test_new, y_pred2))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.7698769146028854
              precision    recall  f1-score   support

           1       0.85      0.71      0.77     19699
           2       0.91      0.74      0.82     20698
           3       0.66      0.79      0.72     17977
           4       0.72      0.80      0.76     18782
           5       0.75      0.82      0.79     17250

    accuracy                           0.77     94406
   macro avg       0.78      0.77      0.77     94406
weighted avg       0.78      0.77      0.77     94406



In [ ]:
joblib.dump(svc, modelfolder+'LinearSVC_category_4000.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/LinearSVC_category_4000.pkl']

###  **Randomforest**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000.pkl')
s=joblib.load(arrayfolder+'s_4000.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)

y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)

y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced')
rfc.fit(X_train,y_new)
y_pred4 = rfc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred4)))
print(classification_report(y_test_new, y_pred4))

Accuracy: 0.6180009745143317
              precision    recall  f1-score   support

           1       0.76      0.50      0.60     19699
           2       0.45      0.92      0.60     20698
           3       0.61      0.58      0.60     17977
           4       0.91      0.56      0.69     18782
           5       0.86      0.50      0.63     17250

    accuracy                           0.62     94406
   macro avg       0.71      0.61      0.62     94406
weighted avg       0.71      0.62      0.62     94406



In [ ]:
joblib.dump(rfc, modelfolder+'RandomForestClassifier_category_4000.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/RandomForestClassifier_category_4000.pkl']

### **XGBRegressor**

In [ ]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sparse matrix package for numeric data.
from scipy import sparse

# Module for word embedding (word2vector)
import gensim

# Module for progress monitoring
import tqdm

# Sklearn
from sklearn.preprocessing import StandardScaler # to standardize features by removing the mean and scaling to unit variance (z=(x-u)/s)
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.svm import SVC # Support Vector Classification
from sklearn.ensemble import RandomForestClassifier # A meta-estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB # Naive Bayes classifier for multinomial models
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # Exhaustive search over specified parameter values for a given estimator
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

import nltk
import re
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn

#XGBoost
from xgboost import XGBRegressor

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

# Joblib
import joblib # To save models

Populating the interactive namespace from numpy and matplotlib


In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000.pkl')
s=joblib.load(arrayfolder+'s_4000.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)

y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)

y_valid_new=[]

for rowv in y_valid:
  if rowv[0] ==1:
    y_valid_new.append(1)
  if rowv[1] ==1:
    y_valid_new.append(2)
  if rowv[2] ==1:
    y_valid_new.append(3)
  if rowv[3] ==1:
    y_valid_new.append(4)
  if rowv[4] ==1:
    y_valid_new.append(5)


y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_valid_new=np.array(y_valid_new)
y_new.shape,y_test_new.shape,y_valid_new.shape

((157343,), (94406,), (78672,))

In [ ]:
my_model = XGBRegressor(n_estimators=10000, learning_rate=0.01, n_jobs=4,subsample = 0.9,
                       min_child_weight = 1,max_depth=4,gamma=1.5,colsample_bytree=0.6 )
my_model.fit(X_train, y_new,
             early_stopping_rounds=90,
             #sample_weight = w,
             eval_set=[(X_valid, y_valid_new)],
             verbose=False)

In [ ]:
joblib.dump(my_model, modelfolder+'XGBRegressorClassifier_category_4000.pkl')

# **Model #2**

### **Load Libraies**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Pillow
!pip install Keras-Preprocessing
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 44.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.

In [3]:
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
import ktrain
from ktrain import text
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences

sns.set()
%matplotlib inline

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [5]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sklearn
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import cross_val_score

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [6]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
# Joblib
import joblib # To save models

In [7]:
datafolder='/content/drive/MyDrive/Community Help /category classification/Data/'
modelfolder='/content/drive/MyDrive/Community Help /category classification/Model/'
arrayfolder='/content/drive/MyDrive/Community Help /category classification/NumPyArrays/'

### **Data Preparion**

In [ ]:
train_data_model2 = pd.read_csv(datafolder+'df_category_model2.csv')
train_data_model2.drop(['Unnamed: 0'],
  axis='columns', inplace=True)
train_data_model2['TITLE']=train_data_model2['TITLE'].fillna("")
train_data_model2.isna().sum()

TITLE                 0
CATEGORY_CLASS        0
CATEGORY_CLASS_NEW    0
dtype: int64

In [ ]:
train_data_model2.head()

,TITLE,CATEGORY_CLASS,CATEGORY_CLASS_NEW
0,Raiders now only team in NFL to go three years...,SPORTS,SPORTS
1,Mike Tyson goes full matrix dodging punches (x...,SPORTS,SPORTS
2,Blake Griffins custom Air Jordans he designed ...,SPORTS,SPORTS
3,How would the Premier League look if clubs cou...,SPORTS,SPORTS
4,"Fernando Alonso, McLaren driver to miss Monaco...",SPORTS,SPORTS


In [ ]:
train_data_model2.shape

(188713, 3)

In [ ]:
X_model2 = []
# the genders
y_model2 = []
# lengths of X elements
X_len_model2 = []
for index, row in train_data_model2.iterrows():
    X_model2.append(row["TITLE"])
    X_len_model2.append([len(row["TITLE"])])
    y_model2.append(row["CATEGORY_CLASS_NEW"])

In [ ]:
joblib.dump(X_len_model2, arrayfolder+'X_len_Classification_model2_category.pkl')
joblib.dump(y_model2, arrayfolder+'y_Classification_model2_category.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_Classification_model2_category.pkl']

#### **Help Methods & Data Cleaning**

In [ ]:
# preprocessing functions
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)

def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)

# remove words that end with one or more identical letters
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)

def remove_allcaps(text):
    caps = re.compile(r'([^a-z0-9()<>\'`\-]){2,}')
    return caps.sub(r'ALLCAPS', text)

def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    return smiley.sub(r'SMILE', text)

def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# tags Part of Speech (POS), because teh lemmatizer needs it
tag_map = defaultdict(lambda : wn.NOUN)
# wn does a grammatical analysis
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# create lemmatizer
word_Lemmatized = WordNetLemmatizer()

def review_to_words(raw_body):
    # remove html tags
    body_text = BeautifulSoup(raw_body).get_text()
    #letters_only = re.sub("[^a-zA-Z]", " ", body_text)
    # lowercase all text
    words = body_text.lower()
    # remove urls
    text = remove_URL(words)
    # remove numbers
    text = remove_number(text)
    # remove smiles
    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
    text = remove_elongated_words(text)
    words = remove_repeat_punct(text)
    # tokenizes and pass to lemmatizer, which lemmatizes taking tags into account (see before)
    words = word_tokenize(words)
    # we don't remove stop words, because doing it on combination with removing the 40 (trial & error estimated parameter) most utilized words (see below) decreases performance
    #stops = set(stopwords.words("english"))
    #meaningful_words = [w for w in words if not w in stops]
    Final_words = []
    for word, tag in pos_tag(words):
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # returns lemmatized texts as strings
    return( " ".join(Final_words))

In [ ]:
clean_train_comments_model2 = [review_to_words(x) for x in X_model2]

<ipython-input-15-1e68d722f9d6>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  body_text = BeautifulSoup(raw_body).get_text()
<ipython-input-15-1e68d722f9d6>:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  body_text = BeautifulSoup(raw_body).get_text()


In [ ]:
joblib.dump(clean_train_comments_model2, arrayfolder+'clean_train_comments_model2_Category_Classification_MLP.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/clean_train_comments_model2_Category_Classification_MLP.pkl']

In [ ]:
joblib.dump(X_len_model2,arrayfolder+'X_len_Classification_model2.pkl')
joblib.dump(y_model2,arrayfolder+'y_Classification_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_Classification_model2.pkl']

### **CountVectorizer**

In [ ]:
clean_train_comments_model2=joblib.load(arrayfolder+'clean_train_comments_model2_Category_Classification_MLP.pkl')

In [ ]:
X_len_model2=joblib.load(arrayfolder+'X_len_Classification_model2.pkl')
y_model2=joblib.load(arrayfolder+'y_Classification_model2.pkl')

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             max_features = 4000)
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments_model2).toarray()


print(train_data_features.shape)
# print vocabulary
vocab = vectorizer.get_feature_names_out()
#print(vocab)
import numpy as np

# counts how many times a word appearco
dist = np.sum(train_data_features, axis=0)

##for tag, count in zip(vocab, dist):
    ##print(count, tag)

    # removes the 40 most utilized words
for _ in range(40):
    index = np.argmax(dist)
    train_data_features = np.delete(train_data_features, index, axis = 1)

train_data_features.shape

# np array
s = np.concatenate((train_data_features,np.array(X_len_model2)),axis = 1)

# 5000 rows (one per author),  and 2000-40+1 (X_len) features
print(s.shape)

# un np.array
y_model2 = np.array(y_model2)

(188713, 4000)
(188713, 3961)


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer2 = CountVectorizer(analyzer = "word",
                             max_features = 3961)
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments_model2).toarray()

In [ ]:
joblib.dump(vectorizer2, arrayfolder+'count_vect_4000_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/count_vect_4000_model2.pkl']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical,plot_model
le=LabelEncoder()
lbls=le.fit_transform(y_model2)
classes=list(le.classes_)
classes=[str(c) for c in classes]
lbls=to_categorical(lbls,num_classes=len(classes))

In [ ]:
classes

['GENERAL-2', 'HEALTH', 'SPORTS', 'WORLD NEWS']

In [ ]:
joblib.dump(classes, arrayfolder+'classes_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/classes_model2.pkl']

In [ ]:
joblib.dump(lbls, arrayfolder+'lbls_MLP_4000_model2.pkl')
joblib.dump(s, arrayfolder+'s_4000_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/s_4000_model2.pkl']

### **MLP Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000_model2.pkl')
s=joblib.load(arrayfolder+'s_4000_model2.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

((169841, 3961), (18872, 3961), (169841, 4), (18872, 4))

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

((135872, 3961), (33969, 3961), (135872, 4), (33969, 4))

In [ ]:
joblib.dump(X_text, arrayfolder+'X_text_MLP.pkl')
joblib.dump(y_text, arrayfolder+'y_text_MLP.pkl')

joblib.dump(X_valid, arrayfolder+'X_valid_MLP.pkl')
joblib.dump(y_valid, arrayfolder+'y_valid_MLP.pkl')

joblib.dump(X_train, arrayfolder+'X_train_MLP.pkl')
joblib.dump(y_train, arrayfolder+'y_train_MLP.pkl')

In [ ]:
# Define MLP Classifier:
## Activation function for the hidden layer: "rectified linear unit function"
## Solver for weight optimization: "stochastic gradient-based optimizer"
## Alpha: regularization parameter
## Learning rate schedule for weight updates: "gradually decreases the learning rate at each time step t using an inverse scaling exponent of power_t"
## Verbose: "True" in order to print progress messages to stdout.
## Early stopping: "True" in order to use early stopping to terminate training when validation score is not improving. It automatically sets aside 10% of training data as validation and terminate training when validation score is not improving by at least tol for n_iter_no_change consecutive epochs.

mlpClf_model2 = MLPClassifier(activation= 'relu', solver = 'adam',
                       alpha = 0.05, learning_rate = 'invscaling', verbose = True,
                       early_stopping = True, max_iter = 400, random_state=0)

# Model fit
mlpClf_model2.fit(X_train, y_train)

Iteration 1, loss = 1.28233850
Validation score: 0.751178
Iteration 2, loss = 0.87278572
Validation score: 0.752576
Iteration 3, loss = 0.82580117
Validation score: 0.778334
Iteration 4, loss = 0.79995818
Validation score: 0.780836
Iteration 5, loss = 0.77925203
Validation score: 0.790845
Iteration 6, loss = 0.76063599
Validation score: 0.805490
Iteration 7, loss = 0.74091714
Validation score: 0.797468
Iteration 8, loss = 0.72036569
Validation score: 0.800559
Iteration 9, loss = 0.70341842
Validation score: 0.805785
Iteration 10, loss = 0.69069961
Validation score: 0.811966
Iteration 11, loss = 0.67629618
Validation score: 0.824772
Iteration 12, loss = 0.66493052
Validation score: 0.823889
Iteration 13, loss = 0.65600077
Validation score: 0.819620
Iteration 14, loss = 0.64560201
Validation score: 0.812114
Iteration 15, loss = 0.63872810
Validation score: 0.820209
Iteration 16, loss = 0.62992119
Validation score: 0.829040
Iteration 17, loss = 0.62431979
Validation score: 0.822049
Iterat

MLPClassifier(alpha=0.05, early_stopping=True, learning_rate='invscaling',
              max_iter=400, random_state=0, verbose=True)

In [ ]:
joblib.dump(mlpClf_model2, modelfolder+'mlpClf_category_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/mlpClf_category_model2.pkl']

In [ ]:
y_score = mlpClf_model2.predict(X_valid)

In [ ]:
roc_auc = roc(y_valid,y_score, multi_class='ovr')
roc_auc

0.9060152379340529

### **LogisticRegression Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000_model2.pkl')
s=joblib.load(arrayfolder+'s_4000_model2.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)

In [ ]:
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

((135872,), (33969,))

In [ ]:
lr = LogisticRegression(C = 3, max_iter = 1000, n_jobs=-1,multi_class='multinomial')
lr.fit(X_train, y_new)
y_pred1 = lr.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred1)))
print(classification_report(y_test_new, y_pred1))

Accuracy: 0.8641113956843004
              precision    recall  f1-score   support

           1       0.71      0.81      0.76      7003
           2       0.88      0.88      0.88      9755
           3       0.95      0.90      0.92      8823
           4       0.91      0.86      0.88      8388

    accuracy                           0.86     33969
   macro avg       0.86      0.86      0.86     33969
weighted avg       0.87      0.86      0.87     33969



In [ ]:
joblib.dump(lr, modelfolder+'LogisticRegression_category_4000_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/LogisticRegression_category_4000_model2.pkl']

###  **Support Vector Machine**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000_model2.pkl')
s=joblib.load(arrayfolder+'s_4000_model2.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)

In [ ]:
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_new)
y_pred2 = svc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred2)))
print(classification_report(y_test_new, y_pred2))
joblib.dump(svc, modelfolder+'LinearSVC_category_4000_model2.pkl')

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.8205716977243958
              precision    recall  f1-score   support

           1       0.81      0.53      0.64      7003
           2       0.79      0.93      0.85      9755
           3       0.95      0.85      0.90      8823
           4       0.77      0.91      0.83      8388

    accuracy                           0.82     33969
   macro avg       0.83      0.80      0.80     33969
weighted avg       0.83      0.82      0.81     33969



['/content/drive/MyDrive/Community Help /category classification/Model/LinearSVC_category_4000_model2.pkl']

###  **Randomforest**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000_model2.pkl')
s=joblib.load(arrayfolder+'s_4000_model2.pkl')

In [ ]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.4, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.5, test_size=0.3,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)

In [ ]:
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced')
rfc.fit(X_train,y_new)
y_pred4 = rfc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred4)))
print(classification_report(y_test_new, y_pred4))
joblib.dump(rfc, modelfolder+'RandomForestClassifier_category_4000_model2.pkl')

Accuracy: 0.7175071388619035
              precision    recall  f1-score   support

           1       0.53      0.51      0.52      7003
           2       0.63      0.82      0.71      9755
           3       0.92      0.79      0.85      8823
           4       0.84      0.70      0.76      8388

    accuracy                           0.72     33969
   macro avg       0.73      0.70      0.71     33969
weighted avg       0.73      0.72      0.72     33969



['/content/drive/MyDrive/Community Help /category classification/Model/RandomForestClassifier_category_4000_model2.pkl']

### **XGBRegressor**

In [8]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sparse matrix package for numeric data.
from scipy import sparse

# Module for word embedding (word2vector)
import gensim

# Module for progress monitoring
import tqdm

# Sklearn
from sklearn.preprocessing import StandardScaler # to standardize features by removing the mean and scaling to unit variance (z=(x-u)/s)
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.svm import SVC # Support Vector Classification
from sklearn.ensemble import RandomForestClassifier # A meta-estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB # Naive Bayes classifier for multinomial models
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # Exhaustive search over specified parameter values for a given estimator
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

import nltk
import re
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn

#XGBoost
from xgboost import XGBRegressor

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

# Joblib
import joblib # To save models

Populating the interactive namespace from numpy and matplotlib


In [9]:
lbls=joblib.load(arrayfolder+'lbls_MLP_4000_model2.pkl')
s=joblib.load(arrayfolder+'s_4000_model2.pkl')

In [10]:
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

((169841, 3961), (18872, 3961), (169841, 4), (18872, 4))

In [11]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

((152856, 3961), (16985, 3961), (152856, 4), (16985, 4))

In [16]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)

y_valid_new=[]

for rowv in y_valid:
  if rowv[0] ==1:
    y_valid_new.append(1)
  if rowv[1] ==1:
    y_valid_new.append(2)
  if rowv[2] ==1:
    y_valid_new.append(3)
  if rowv[3] ==1:
    y_valid_new.append(4)


In [14]:
y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

((152856,), (16985,))

In [17]:
my_model = XGBRegressor(n_estimators=10000, learning_rate=0.01, n_jobs=4,subsample = 0.9,
                       min_child_weight = 1,max_depth=4,gamma=1.5,colsample_bytree=0.6 )
my_model.fit(X_train, y_new,
             early_stopping_rounds=90,
             #sample_weight = w,
             eval_set=[(X_valid, y_valid_new)],
             verbose=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1.5, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=4, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
joblib.dump(my_model, modelfolder+'XGBRegressorClassifier_category_4000_model2.pkl')

['/content/drive/MyDrive/Community Help /category classification/Model/XGBRegressorClassifier_category_4000_model2.pkl']

# **Model #3**

### **Load Libraies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Pillow
!pip install Keras-Preprocessing
!pip install ktrain

In [ ]:
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
import ktrain
from ktrain import text
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences

sns.set()
%matplotlib inline

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sklearn
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import cross_val_score

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
# Joblib
import joblib # To save models

In [ ]:
datafolder='/content/drive/MyDrive/Community Help /category classification/Data/'
modelfolder='/content/drive/MyDrive/Community Help /category classification/Model/'
arrayfolder='/content/drive/MyDrive/Community Help /category classification/NumPyArrays/'

### **Data Preparion**

In [ ]:
train_data_model3 = pd.read_csv(datafolder+'df_category_model3.csv')
train_data_model3.drop(['Unnamed: 0'],
  axis='columns', inplace=True)
train_data_model3['TITLE']=train_data_model3['TITLE'].fillna("")
train_data_model3.isna().sum()

TITLE                 0
CATEGORY_CLASS        0
CATEGORY_CLASS_NEW    0
dtype: int64

In [ ]:
train_data_model3.head()

,TITLE,CATEGORY_CLASS,CATEGORY_CLASS_NEW
0,There Were 2 Mass Shootings In Texas Last Week...,CRIME,CRIME
1,Rachel Dolezal Faces Felony Charges For Welfar...,CRIME,CRIME
2,Morgan Freeman Dropped From Marketing Campaign...,WOMEN,WOMEN
3,"Man Faces Charges After Pulling Knife, Stun Gu...",CRIME,CRIME
4,2 People Injured In Indiana School Shooting,CRIME,CRIME


In [ ]:
train_data_model3.shape

(38905, 3)

In [ ]:
X_model3 = []
# the genders
y_model3 = []
# lengths of X elements
X_len_model3 = []
for index, row in train_data_model3.iterrows():
    X_model3.append(row["TITLE"])
    X_len_model3.append([len(row["TITLE"])])
    y_model3.append(row["CATEGORY_CLASS_NEW"])

In [ ]:
joblib.dump(X_len_model3, arrayfolder+'X_len_Classification_model3_category.pkl')
joblib.dump(y_model3, arrayfolder+'y_Classification_model3_category.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_Classification_model3_category.pkl']

#### **Help Methods & Data Cleaning**

In [ ]:
# preprocessing functions
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)

def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)

# remove words that end with one or more identical letters
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)

def remove_allcaps(text):
    caps = re.compile(r'([^a-z0-9()<>\'`\-]){2,}')
    return caps.sub(r'ALLCAPS', text)

def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    return smiley.sub(r'SMILE', text)

def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# tags Part of Speech (POS), because teh lemmatizer needs it
tag_map = defaultdict(lambda : wn.NOUN)
# wn does a grammatical analysis
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# create lemmatizer
word_Lemmatized = WordNetLemmatizer()

def review_to_words(raw_body):
    # remove html tags
    body_text = BeautifulSoup(raw_body).get_text()
    #letters_only = re.sub("[^a-zA-Z]", " ", body_text)
    # lowercase all text
    words = body_text.lower()
    # remove urls
    text = remove_URL(words)
    # remove numbers
    text = remove_number(text)
    # remove smiles
    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
    text = remove_elongated_words(text)
    words = remove_repeat_punct(text)
    # tokenizes and pass to lemmatizer, which lemmatizes taking tags into account (see before)
    words = word_tokenize(words)
    # we don't remove stop words, because doing it on combination with removing the 40 (trial & error estimated parameter) most utilized words (see below) decreases performance
    #stops = set(stopwords.words("english"))
    #meaningful_words = [w for w in words if not w in stops]
    Final_words = []
    for word, tag in pos_tag(words):
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # returns lemmatized texts as strings
    return( " ".join(Final_words))

In [ ]:
clean_train_comments_model3 = [review_to_words(x) for x in X_model3]

<ipython-input-15-1e68d722f9d6>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  body_text = BeautifulSoup(raw_body).get_text()


In [ ]:
joblib.dump(clean_train_comments_model3, arrayfolder+'clean_train_comments_model3_Category_Classification_MLP.pkl')
joblib.dump(X_len_model3,arrayfolder+'X_len_Classification_model3.pkl')
joblib.dump(y_model3,arrayfolder+'y_Classification_model3.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/y_Classification_model3.pkl']

### **CountVectorizer**

In [ ]:
clean_train_comments_model3=joblib.load(arrayfolder+'clean_train_comments_model3_Category_Classification_MLP.pkl')
X_len_model3=joblib.load(arrayfolder+'X_len_Classification_model3.pkl')
y_model3=joblib.load(arrayfolder+'y_Classification_model3.pkl')


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             max_features = 10000)
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments_model3).toarray()


print(train_data_features.shape)
# print vocabulary
vocab = vectorizer.get_feature_names_out()
#print(vocab)
import numpy as np

# counts how many times a word appearco
dist = np.sum(train_data_features, axis=0)

##for tag, count in zip(vocab, dist):
    ##print(count, tag)

    # removes the 40 most utilized words
for _ in range(40):
    index = np.argmax(dist)
    train_data_features = np.delete(train_data_features, index, axis = 1)

train_data_features.shape

# np array
s = np.concatenate((train_data_features,np.array(X_len_model3)),axis = 1)

# 5000 rows (one per author),  and 2000-40+1 (X_len) features
print(s.shape)

# un np.array
y_model3 = np.array(y_model3)

(38905, 10000)
(38905, 9961)


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer3 = CountVectorizer(analyzer = "word",
                             max_features = 9961)
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments_model3).toarray()

In [ ]:
joblib.dump(vectorizer3, arrayfolder+'count_vect_10000_model3.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/count_vect_10000_model3.pkl']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical,plot_model
le=LabelEncoder()
lbls=le.fit_transform(y_model3)
classes=list(le.classes_)
classes=[str(c) for c in classes]
lbls=to_categorical(lbls,num_classes=len(classes))
classes

['ARTS & CULTURE',
 'CRIME',
 'EDUCATION',
 'GREEN',
 'RELIGION',
 'STYLE',
 'TASTE',
 'TRAVEL',
 'WOMEN']

In [ ]:
joblib.dump(classes, arrayfolder+'classes_model3.pkl')
joblib.dump(lbls, arrayfolder+'lbls_MLP_10000_model3.pkl')
joblib.dump(s, arrayfolder+'s_10000_model3.pkl')

['/content/drive/MyDrive/Community Help /category classification/NumPyArrays/s_10000_model3.pkl']

### **MLP Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_10000_model3.pkl')
s=joblib.load(arrayfolder+'s_10000_model3.pkl')

# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape


((35014, 9961), (3891, 9961), (35014, 9), (3891, 9))

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape


((31512, 9961), (3502, 9961), (31512, 9), (3502, 9))

In [ ]:
# Define MLP Classifier:
## Activation function for the hidden layer: "rectified linear unit function"
## Solver for weight optimization: "stochastic gradient-based optimizer"
## Alpha: regularization parameter
## Learning rate schedule for weight updates: "gradually decreases the learning rate at each time step t using an inverse scaling exponent of power_t"
## Verbose: "True" in order to print progress messages to stdout.
## Early stopping: "True" in order to use early stopping to terminate training when validation score is not improving. It automatically sets aside 10% of training data as validation and terminate training when validation score is not improving by at least tol for n_iter_no_change consecutive epochs.

mlpClf_model3 = MLPClassifier(activation= 'relu', solver = 'adam',
                       alpha = 0.05, learning_rate = 'invscaling', verbose = True,
                       early_stopping = True, max_iter = 400, random_state=0)

# Model fit
mlpClf_model3.fit(X_train, y_train)
joblib.dump(mlpClf_model3, modelfolder+'mlpClf_category_model3.pkl')

Iteration 1, loss = 3.34463188
Validation score: 0.054886
Iteration 2, loss = 2.34152085
Validation score: 0.242703
Iteration 3, loss = 1.87604217
Validation score: 0.390546
Iteration 4, loss = 1.66017717
Validation score: 0.482551
Iteration 5, loss = 1.53904440
Validation score: 0.482551
Iteration 6, loss = 1.45817944
Validation score: 0.519036
Iteration 7, loss = 1.40014632
Validation score: 0.516497
Iteration 8, loss = 1.35398844
Validation score: 0.526015
Iteration 9, loss = 1.31214585
Validation score: 0.548858
Iteration 10, loss = 1.27684936
Validation score: 0.551079
Iteration 11, loss = 1.24759907
Validation score: 0.555838
Iteration 12, loss = 1.22064431
Validation score: 0.564721
Iteration 13, loss = 1.19680744
Validation score: 0.559645
Iteration 14, loss = 1.17606630
Validation score: 0.562500
Iteration 15, loss = 1.14706557
Validation score: 0.565355
Iteration 16, loss = 1.12819675
Validation score: 0.572970
Iteration 17, loss = 1.10950062
Validation score: 0.571701
Iterat

['/content/drive/MyDrive/Community Help /category classification/Model/mlpClf_category_model3.pkl']

In [ ]:
y_score = mlpClf_model3.predict(X_valid)

roc_auc = roc(y_valid,y_score, multi_class='ovr')
roc_auc

0.8011522471590584

### **LogisticRegression Classifier**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_10000_model3.pkl')
s=joblib.load(arrayfolder+'s_10000_model3.pkl')

# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape


In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape


In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)
  if rowv[5] ==1:
    y_new.append(6)
  if rowv[6] ==1:
    y_new.append(7)
  if rowv[7] ==1:
    y_new.append(8)
  if rowv[8] ==1:
    y_new.append(9)


y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)
  if rowv[5] ==1:
    y_test_new.append(6)
  if rowv[6] ==1:
    y_test_new.append(7)
  if rowv[7] ==1:
    y_test_new.append(8)
  if rowv[8] ==1:
    y_test_new.append(9)

y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

((31512,), (3502,))

In [ ]:
lr = LogisticRegression(C = 3, max_iter = 1000, n_jobs=-1,multi_class='multinomial')
lr.fit(X_train, y_new)
y_pred1 = lr.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred1)))
print(classification_report(y_test_new, y_pred1))
joblib.dump(lr, modelfolder+'LogisticRegression_category_10000_model3.pkl')

Accuracy: 0.6976013706453456
              precision    recall  f1-score   support

           1       0.63      0.65      0.64       453
           2       0.79      0.77      0.78       454
           3       0.73      0.71      0.72       171
           4       0.76      0.73      0.74       418
           5       0.72      0.66      0.69       389
           6       0.60      0.67      0.63       338
           7       0.75      0.73      0.74       360
           8       0.72      0.70      0.71       343
           9       0.64      0.68      0.66       576

    accuracy                           0.70      3502
   macro avg       0.70      0.70      0.70      3502
weighted avg       0.70      0.70      0.70      3502



['/content/drive/MyDrive/Community Help /category classification/Model/LogisticRegression_category_10000_model3.pkl']

###  **Support Vector Machine**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_10000_model3.pkl')
s=joblib.load(arrayfolder+'s_10000_model3.pkl')
# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape


In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)
  if rowv[5] ==1:
    y_new.append(6)
  if rowv[6] ==1:
    y_new.append(7)
  if rowv[7] ==1:
    y_new.append(8)
  if rowv[8] ==1:
    y_new.append(9)


y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)
  if rowv[5] ==1:
    y_test_new.append(6)
  if rowv[6] ==1:
    y_test_new.append(7)
  if rowv[7] ==1:
    y_test_new.append(8)
  if rowv[8] ==1:
    y_test_new.append(9)

y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_new)
y_pred2 = svc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred2)))
print(classification_report(y_test_new, y_pred2))
joblib.dump(svc, modelfolder+'LinearSVC_category_10000_model3.pkl')

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.6616219303255283
              precision    recall  f1-score   support

           1       0.65      0.58      0.61       453
           2       0.71      0.83      0.77       454
           3       0.71      0.75      0.73       171
           4       0.85      0.60      0.70       418
           5       0.64      0.73      0.68       389
           6       0.54      0.69      0.61       338
           7       0.58      0.79      0.67       360
           8       0.60      0.73      0.66       343
           9       0.79      0.43      0.55       576

    accuracy                           0.66      3502
   macro avg       0.68      0.68      0.66      3502
weighted avg       0.68      0.66      0.66      3502



['/content/drive/MyDrive/Community Help /category classification/Model/LinearSVC_category_10000_model3.pkl']

###  **Randomforest**

In [ ]:
lbls=joblib.load(arrayfolder+'lbls_MLP_10000_model3.pkl')
s=joblib.load(arrayfolder+'s_10000_model3.pkl')

# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)
  if rowv[5] ==1:
    y_new.append(6)
  if rowv[6] ==1:
    y_new.append(7)
  if rowv[7] ==1:
    y_new.append(8)
  if rowv[8] ==1:
    y_new.append(9)


y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)
  if rowv[5] ==1:
    y_test_new.append(6)
  if rowv[6] ==1:
    y_test_new.append(7)
  if rowv[7] ==1:
    y_test_new.append(8)
  if rowv[8] ==1:
    y_test_new.append(9)

y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced')
rfc.fit(X_train,y_new)
y_pred4 = rfc.predict(X_text)
print("Accuracy: "+str(accuracy_score(y_test_new,y_pred4)))
print(classification_report(y_test_new, y_pred4))
joblib.dump(rfc, modelfolder+'RandomForestClassifier_category_10000_model3.pkl')

Accuracy: 0.5471159337521416
              precision    recall  f1-score   support

           1       0.48      0.40      0.44       453
           2       0.66      0.72      0.69       454
           3       0.55      0.74      0.63       171
           4       0.67      0.45      0.54       418
           5       0.86      0.51      0.64       389
           6       0.29      0.69      0.41       338
           7       0.55      0.65      0.60       360
           8       0.61      0.50      0.55       343
           9       0.69      0.43      0.53       576

    accuracy                           0.55      3502
   macro avg       0.60      0.57      0.56      3502
weighted avg       0.61      0.55      0.56      3502



['/content/drive/MyDrive/Community Help /category classification/Model/RandomForestClassifier_category_10000_model3.pkl']

### **XGBRegressor**

In [19]:
# Numpy & matplotlib for notebooks
%pylab inline

# Pandas for data analysis and manipulation
import pandas as pd

# Sparse matrix package for numeric data.
from scipy import sparse

# Module for word embedding (word2vector)
import gensim

# Module for progress monitoring
import tqdm

# Sklearn
from sklearn.preprocessing import StandardScaler # to standardize features by removing the mean and scaling to unit variance (z=(x-u)/s)
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron classifier which optimizes the log-loss function using LBFGS or sdg.
from sklearn.svm import SVC # Support Vector Classification
from sklearn.ensemble import RandomForestClassifier # A meta-estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting
from sklearn.decomposition import PCA, TruncatedSVD # Principal component analysis (PCA); dimensionality reduction using truncated SVD.
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB # Naive Bayes classifier for multinomial models
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score as roc # Compute Area Under the Receiver Operating Characteristic Curve from prediction scores
from sklearn.metrics import roc_curve, auc # Compute ROC; Compute Area Under the Curve (AUC) using the trapezoidal rule
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # Exhaustive search over specified parameter values for a given estimator
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation
from sklearn.model_selection import train_test_split # to split arrays or matrices into random train and test subsets
from sklearn.model_selection import KFold # K-Folds cross-validator providing train/test indices to split data in train/test sets.
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

import nltk
import re
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn

#XGBoost
from xgboost import XGBRegressor

# Matplotlib
import matplotlib # Data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Seaborn
import seaborn as sns # Statistical data visualization (based on matplotlib)

# Joblib
import joblib # To save models

Populating the interactive namespace from numpy and matplotlib


In [20]:
lbls=joblib.load(arrayfolder+'lbls_MLP_10000_model3.pkl')
s=joblib.load(arrayfolder+'s_10000_model3.pkl')

# Split the data for training
X1, X_valid, Y1, y_valid = train_test_split(s, lbls,
                                                      train_size=0.9, test_size=0.1,
                                                      random_state=0,stratify = lbls)
X1.shape,X_valid.shape,Y1.shape,y_valid.shape

((35014, 9961), (3891, 9961), (35014, 9), (3891, 9))

In [ ]:
# Split the data for training
X_train, X_text, y_train, y_text = train_test_split(X1, Y1,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0,stratify = Y1)
X_train.shape,X_text.shape,y_train.shape,y_text.shape

In [ ]:
y_new=[]
for rowv in y_train:
  if rowv[0] ==1:
    y_new.append(1)
  if rowv[1] ==1:
    y_new.append(2)
  if rowv[2] ==1:
    y_new.append(3)
  if rowv[3] ==1:
    y_new.append(4)
  if rowv[4] ==1:
    y_new.append(5)
  if rowv[5] ==1:
    y_new.append(6)
  if rowv[6] ==1:
    y_new.append(7)
  if rowv[7] ==1:
    y_new.append(8)
  if rowv[8] ==1:
    y_new.append(9)


y_test_new=[]
for rowv in y_text:
  if rowv[0] ==1:
    y_test_new.append(1)
  if rowv[1] ==1:
    y_test_new.append(2)
  if rowv[2] ==1:
    y_test_new.append(3)
  if rowv[3] ==1:
    y_test_new.append(4)
  if rowv[4] ==1:
    y_test_new.append(5)
  if rowv[5] ==1:
    y_test_new.append(6)
  if rowv[6] ==1:
    y_test_new.append(7)
  if rowv[7] ==1:
    y_test_new.append(8)
  if rowv[8] ==1:
    y_test_new.append(9)

y_new=np.array(y_new)
y_test_new=np.array(y_test_new)
y_new.shape,y_test_new.shape

In [ ]:
y_valid_new=[]
for rowv in y_valid:
  if rowv[0] ==1:
    y_valid_new.append(1)
  if rowv[1] ==1:
    y_valid_new.append(2)
  if rowv[2] ==1:
    y_valid_new.append(3)
  if rowv[3] ==1:
    y_valid_new.append(4)
  if rowv[4] ==1:
    y_valid_new.append(5)
  if rowv[5] ==1:
    y_valid_new.append(6)
  if rowv[6] ==1:
    y_valid_new.append(7)
  if rowv[7] ==1:
    y_valid_new.append(8)
  if rowv[8] ==1:
    y_valid_new.append(9)

In [ ]:
my_model = XGBRegressor(n_estimators=10000, learning_rate=0.01, n_jobs=4,subsample = 0.9,
                       min_child_weight = 1,max_depth=4,gamma=1.5,colsample_bytree=0.6 )
my_model.fit(X_train, y_new,
             early_stopping_rounds=90,
             #sample_weight = w,
             eval_set=[(X_valid, y_valid_new)],
             verbose=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
joblib.dump(my_model, modelfolder+'XGBRegressorClassifier_category_10000_model3.pkl')